In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import scipy.stats as ss
import pingouin as pg

In [3]:
tit = pd.read_csv(Path.cwd() / 'data' / 'titanic.csv')
tit.head(), tit.shape

(   Class    Age   Sex Survived
 0  First  Adult  Male      Yes
 1  First  Adult  Male      Yes
 2  First  Adult  Male      Yes
 3  First  Adult  Male      Yes
 4  First  Adult  Male      Yes,
 (2201, 4))

In [4]:
pd.crosstab(tit['Survived'], tit['Sex'], margins=True)

Sex,Female,Male,All
Survived,,,
No,126,1364,1490
Yes,344,367,711
All,470,1731,2201


In [5]:
pd.crosstab(tit['Survived'], tit['Class'], margins=True)

Class,Crew,First,Second,Third,All
Survived,,,,,
No,673,122,167,528,1490
Yes,212,203,118,178,711
All,885,325,285,706,2201


In [6]:
pd.crosstab(tit['Survived'], tit['Age'], margins=True)

Age,Adult,Child,All
Survived,,,
No,1438,52,1490
Yes,654,57,711
All,2092,109,2201


In [7]:
tit['Age'] = tit['Age'].replace({'Child':0, 'Adult':1})
tit['Sex'] = tit['Sex'].replace({'Male':0, 'Female':1})
tit['Class'] = tit['Class'].replace({'First':1, 'Second':2, 'Third':3, 'Crew':4})

In [8]:
tit.head()

,Class,Age,Sex,Survived
0,1,1,0,Yes
1,1,1,0,Yes
2,1,1,0,Yes
3,1,1,0,Yes
4,1,1,0,Yes
